In [ ]:
from PIL import Image
def convert_file_to_binary(path):
    with open(path, 'rb') as f:
        binary_data = f.read()
        binary_string = ''.join(format(byte, '08b') for byte in binary_data)
    print(f'Binary length : {8*len(binary_string)}')
    return binary_string


In [ ]:
def create_image(name_of_file,img_index,current_index,binary_string):
    # new_image = Image.new('RGB', (7680, 4320), (256, 256, 256))
    #change to 480p)
    new_image = Image.new('RGB', (640, 480), (256, 256, 256))
    # Get the size of the image
    width, height = new_image.size

    # Loop over each pixel in the image and set its value
    # current_index=int(current_index)
    for x in range(width):
        for y in range(height):
            # Set the color of the pixel
            all_pixels={'Red':0,'Green':0,'Blue':0}
            if(len(binary_string)-current_index>=24):
                for i in range(3):
                    binary_substring=binary_string[current_index:current_index+8]
                    current_index+=8
                    decimal_temp = int(binary_substring, 2)
                    all_pixels[list(all_pixels.keys())[i]]=decimal_temp
            elif(len(binary_string)-current_index==16):
                for i in range(2):
                    binary_substring=binary_string[current_index:current_index+8]
                    current_index+=8
                    decimal_temp = int(binary_substring, 2)
                    all_pixels[list(all_pixels.keys())[i+1]]=decimal_temp
            elif(len(binary_string)-current_index==8):
                binary_substring=binary_string[current_index:current_index+8]
                current_index+=8
                decimal_temp = int(binary_substring, 2)
                all_pixels[list(all_pixels.keys())[2]]=decimal_temp
                    
            new_image.putpixel((x, y), (all_pixels['Red'], all_pixels['Green'], all_pixels['Blue']))  
    print("image created")
    # Save the new image
    new_image.save('./images/'+name_of_file+'_pixels_'+'{:04d}'.format(img_index)+'.jpg')
    return current_index
    

In [ ]:
def Create_images(name_of_file,binary_string):
    print("Creating images...")
    total_length=len(binary_string)
    img_index=1
    current_index=0
    
    while(total_length>307200):
        print(f'Creating image {img_index}')
        current_index=create_image(name_of_file,img_index,current_index,binary_string)
        print(f"{current_index} out of {total_length}")
        img_index+=1
        if(current_index==total_length):
            break
    if(current_index<total_length):
        print(f'Creating image {img_index}')
        create_image(name_of_file,img_index,current_index,binary_string)
    # while(total_length>33177600):
    #     current_index=create_image(name_of_file,img_index,current_index,binary_string)
    #     img_index+=1
        
    # if(current_index<33177600):
    #     create_image(name_of_file,img_index,current_index,binary_string)

In [ ]:
def cal_no_of_images(name_of_file):
    import os
    directory = './images/'
    file_list = os.listdir(directory)
    num_files=0
    for file in file_list:
        if name_of_file == str(file)[:-16]:
            print("images are : "+str(file))
            num_files+=1

    return num_files


In [ ]:
def CreateVideo(name_of_file):
    print("Creating video...")
    import imageio.v2
    writer = imageio.v2.get_writer('./videos/'+name_of_file+'.mp4', fps=30)
    noi=cal_no_of_images(name_of_file)
    for i in range(noi):
        print("Video: images are : "+str(i))
        image = imageio.v2.imread('./images/'+name_of_file+'_pixels_'+'{:04d}'.format(i+1)+'.jpg')
        writer.append_data(image)

    writer.close()


In [ ]:
def Delete_Images(name_of_file):
    import os
    directory = './images/'
    file_list = os.listdir(directory)
    for file in file_list:
        if name_of_file == str(file)[:-16]:
            os.remove('./images/'+file)
            print("Deleted : "+str(file))

In [ ]:
def convertdatatovideo(name_of_file,path):
    binary_string=convert_file_to_binary(path)
    Create_images(name_of_file,binary_string)
    CreateVideo(name_of_file)
    Delete_Images(name_of_file)
    

In [ ]:
# dot_index = s.rfind('.')
import os
directory = './inputs/'
file_list = os.listdir(directory)
for file in file_list:
    convertdatatovideo(str(file),directory+str(file))